In [116]:
from stock_data_ingestion import ingest_stocks_to_df
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display 
from keras.layers import LSTM, Dense, Input
from keras import Sequential
from sklearn.model_selection import TimeSeriesSplit
from keras.optimizers import Adam, SGD
from keras.losses import MeanSquaredError
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.preprocessing.sequence import TimeseriesGenerator

## Data Preprocessing

In [103]:
## comment first line out after first use to prevent 
## giving the source website for the stocks too much traffic. 
# stocks_df = ingest_stocks_to_df()
# stock_list = stocks_df.company_name.unique()
## displays
# display(stocks_df)
# for stock in stock_list:
#     print(stock, 'len', len(stocks_df[stocks_df.company_name == stock].index))
# sns.lineplot(x=stocks_df.index, y='Adj Close',data=stocks_df, legend='auto', hue='company_name')
# plt.show()

In [133]:
def get_data_targets(stock_arr, sequence_inlength):
    """where stock_arr is sequential np.array and 
    sequence_inlength is the length of the data sequences"""
    x, y = list(), list()
    for index in range(sequence_inlength, len(stock_arr)):
        window = stock_arr[index-sequence_inlength:index]
        x.append(window)
        y.append([stock_arr[index]])
    data = np.array(x)
    targets = np.array(y)
    return data, targets
def to_ts_data(stock=stock_list[0], sequence_inlength=60):
    stock_arr = stocks_df[stocks_df.company_name == stock]['Adj Close'].to_numpy()
    data, targets = get_data_targets(stock_arr, sequence_inlength)
#     display(data, targets)
    data_gen = TimeseriesGenerator(data, targets, length=sequence_inlength)
    ##shape (number of batches, input number of steps before prediction, feature-dimensions for input)
    return data_gen

## Model Building

In [134]:
def lstm_model(nodes_lstm=50, dropout=0.0, recurrent_dropout=0.0, learning_rate=0.01, loss='mse', optimizer=Adam, metrics=['mse']):
    lstm_layer = LSTM(nodes_lstm, dropout=dropout, recurrent_dropout=recurrent_dropout)
    model = Sequential()
#     model.add(Input((1,1)))
    model.add(lstm_layer)
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer(learning_rate=learning_rate),
              loss=loss,
              metrics=metrics)
    return model
# lstm_model().summary()

In [137]:
data_gen = to_ts_data() 
display(data_gen, len(data_gen), data_gen[0])
kr = KerasRegressor(build_fn=lstm_model())
display(kr.get_params())
kr.fit(data_gen)

11

(array([[[5.73000002, 5.71000004, 5.76999998, ..., 4.9000001 ,
          4.80999994, 4.80000019],
         [5.71000004, 5.76999998, 5.80000019, ..., 4.80999994,
          4.80000019, 4.75      ],
         [5.76999998, 5.80000019, 5.76000023, ..., 4.80000019,
          4.75      , 4.86999989],
         ...,
         [4.9000001 , 4.80999994, 4.80000019, ..., 5.01999998,
          5.        , 5.        ],
         [4.80999994, 4.80000019, 4.75      , ..., 5.        ,
          5.        , 5.        ],
         [4.80000019, 4.75      , 4.86999989, ..., 5.        ,
          5.        , 5.        ]],
 
        [[5.71000004, 5.76999998, 5.80000019, ..., 4.80999994,
          4.80000019, 4.75      ],
         [5.76999998, 5.80000019, 5.76000023, ..., 4.80000019,
          4.75      , 4.86999989],
         [5.80000019, 5.76000023, 5.84000015, ..., 4.75      ,
          4.86999989, 4.98000002],
         ...,
         [4.80999994, 4.80000019, 4.75      , ..., 5.        ,
          5.        , 5.

{'build_fn': <tensorflow.python.keras.engine.sequential.Sequential at 0x7f89d2c91a90>}

TypeError: fit() missing 1 required positional argument: 'y'